In [1]:
import requests
import time
import random
from openpyxl import Workbook
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen

UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]
headers = {'user-agent': ua}

In [2]:
def get_urls():
    """从首页获取文章url"""
    req = Request(url='https://www.dermstore.com/blog/category/skin/', headers = headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features='lxml')
    all_href= soup.find_all('meta', itemprop ='url')
    b_list = []
    for l in all_href:
        b_list.append(l['content'])
    return b_list

In [3]:
def get_info(url):
    """获取标题、作者、发布时间及正文内容"""
    time.sleep(0.1)
    req = Request(url, headers = headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features='lxml')
    if soup.find('h1', class_='postTitle') == None:
        return {'url': url, 'title': None, 'author and date': None, 'body': None}
    else:
        title = soup.find('h1', class_='postTitle').text
        author_date = soup.find('p',class_='postDetails' ).text
        p_list = []
        for p in soup.find_all('p', style='text-align: justify;'):
            p_list.append(p.text)
        body = '\n'.join(p_list)
        return {'url': url, 'title': title, 'author and date': author_date, 'body': body}

In [4]:
def save_as_xls(info_list):
    """将获取到的文章信息存储到本地"""
    workbook = Workbook()
    worksheet = workbook.active
    worksheet.title = 'sheet1'
    titles = ['URL', 'Title', 'Author and Date', 'Content']
    keys = ['url', 'title', 'author and date', 'body']
    for index, title in enumerate(titles, start=1):
        worksheet.cell(row=1, column=index, value=title)
    for i, info in enumerate(info_list, start=1):
        for j,key in enumerate(keys, start=1):
            worksheet.cell(row=i+1, column=j, value=info[key])
    workbook.save('info.xlsx')

In [8]:
def main():
    """main function"""
    info_list = []
    for url in get_urls():
        info_list.append(get_info(url))
    save_as_xls(info_list)
    
if __name__ == "__main__":
    main()

In [172]:
get_info("https://www.dermstore.com/blog/skin-care-ingredients-dictionary/")

{'url': 'https://www.dermstore.com/blog/skin-care-ingredients-dictionary/',
 'title': 'Dermstore’s Guide to Beauty Ingredients',
 'author and date': 'BY Dermstore Editors · March 21, 2019',
 'body': 'Short Definition\nAcai is a deep purple, antioxidant-filled berry from Acai Palm Tree.\nLong Definition\nPacked with more antioxidants than cranberries, raspberries, blackberries, strawberries or blueberries, acai is a potent source of the antioxidants ferulic acid, epicatechin and vitamins A, B, C and E, which is why it’s used often in anti-aging skin care products. It also contains omega-3 and omega-6 fatty acids, which are important building blocks for the skin and hair.\nBack to Top\nShort Definition\nAdenosine is derived from yeast and helps to smooth the skin.\nLong Definition\nNaturally present in living things, adenosine is an important source of energy required for normal function and growth of cells. When used regularly in skin care products, adenosine has been shown to help supp